## Spark 101 Exercises
### Corey Solitaire
`11.24.2020`

In [11]:
import pyspark
import pandas as pd
import numpy as np

from pydataset import data
from vega_datasets import data

from pyspark.sql.functions import col, expr, concat, sum, avg, min, max, count, mean, round
from pyspark.sql.functions import lit, regexp_extract, regexp_replace, when,asc, desc, month, year, quarter  

***
## Method to create spark session

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

***
## Reference Material

In [3]:
pd_v_spark = pd.DataFrame([['pd.read_csv("myfile.csv")', 
                            'spark.read.load("myfile.csv", format = "csv", sep = ",")'], 
                           ['pd.read_json("myfile.json")', 
                            'spark.read.load("myfile.json", format = "json") OR spark.read.json("myfile.json")']], 
                          index = ['csv', 'json'], 
                          columns = ['pandas', 'spark'])

pd_v_spark = pd_v_spark.append(pd.DataFrame([['pd_df.head()', 'sp_df.show(), .head(), .take()'],
                                             ['pd_df.head(1)', 'sp_df.first()'],
                                             ['pd_df.describe()', 'sp_df.describe()'],
                                             ['pd_df.columns', 'sp_df.columns'],
                                             ['len(pd_df)', 'sp_df.count()'],
                                             ['len(pd_df.drop_duplicates())', 'sp_df.distinct().count()'],
                                             ['pd_df.info()', 'sp_df.printSchema()']
                                            ],
                                            index = ['1st n rows', '1st row','summary statistics', 
                                                     'column names', '# rows', '# distinct rows', 
                                                     'df schema info'], 
                                            columns = ['pandas', 'spark']))

pd_v_spark = pd_v_spark.append(pd.DataFrame([['pd_df[["col1", "col2"]]', 
                                              'sp_df.select(sp_df.col1, sp_df.col2)']
                                            ],
                                            index = ['select columns'], 
                                            columns = ['pandas', 'spark']))

pd_v_spark = pd_v_spark.append(pd.DataFrame([['pd_df[pd_df.c1 > 0]', 'sp_df.filter(df.c1 > 0), sp_df.where(df.c1 > 0)'],
                                            ],
                                            index = ['conditional filtering'], 
                                            columns = ['pandas', 'spark']))

pd_v_spark = pd_v_spark.append(pd.DataFrame([['np.where(pd_df.c1.array > 0, "positive")', 
                                              'sp_df.select(df.c1, when(df.c1 > 0, "positive").alias("number_sign"))'],
                                            ],
                                            index = ['conditional assigning'], 
                                            columns = ['pandas', 'spark']))

pd_v_spark = pd_v_spark.append(pd.DataFrame([['np.where(pd_df.c1.array > 0, "pos", "neg")', 
                                              'sp_df.select(df.c1, when(df.c1 > 0, "pos").otherwise("neg").alias("number_sign"))'],
                                            ],
                                            index = ['conditional assigning with else'], 
                                            columns = ['pandas', 'spark']))


pd_v_spark = pd_v_spark.append(pd.DataFrame([['pd_df.sort_values(by=["c1"])', 
                                              'sp_df.sort(sp_df.c1)'],
                                             ['pd_df.sort_values(by=["c1","c2"])',
                                              'sp_df.sort(sp_df.c1, sp_df.c2)'],
                                             ['pd_df.sort_values(by=["c1","c2"], ascending=[False, True])',
                                              'sp_df.sort(sp_df.c1.desc(), sp_df.c2)'],
                                             ['pd_df.sort_values(by=["c1","c2"], ascending=False)', 
                                              'sp_df.sort(desc("c1"), desc("c2")) OR sp_df.sort(col("c1").desc(), col("c2").desc())']
                                            ],
                                            index = ['sort 1 col asc', 'sort 2+ cols asc', 'sort 2+ cols desc/asc', 'sort 2+ cols desc'], 
                                            columns = ['pandas', 'spark']))
pd_v_spark

,pandas,spark
csv,"pd.read_csv(""myfile.csv"")","spark.read.load(""myfile.csv"", format = ""csv"", sep = "","")"
json,"pd.read_json(""myfile.json"")","spark.read.load(""myfile.json"", format = ""json"") OR spark.read.json(""myfile.json"")"
1st n rows,pd_df.head(),"sp_df.show(), .head(), .take()"
1st row,pd_df.head(1),sp_df.first()
summary statistics,pd_df.describe(),sp_df.describe()
column names,pd_df.columns,sp_df.columns
# rows,len(pd_df),sp_df.count()
# distinct rows,len(pd_df.drop_duplicates()),sp_df.distinct().count()
df schema info,pd_df.info(),sp_df.printSchema()
select columns,"pd_df[[""col1"", ""col2""]]","sp_df.select(sp_df.col1, sp_df.col2)"


***
## 1. Create a spark data frame that contains your favorite programming languages.

### -The name of the column should be language

In [4]:
# Create Dataframe
lang_df = pd.DataFrame([['yes', 'no', 'yes', 'no'], 
                        ['yes', 'yes', 'yes', 'yes']], 
                        index = ['csv', 'json'], 
                        columns = ['python', 'javascript', 'r', 'java'])

# to display and see all text in dataframe
pd.set_option('display.max_colwidth', 10000)


lang_df

,python,javascript,r,java
csv,yes,no,yes,no
json,yes,yes,yes,yes


In [5]:
# Convert to spark
sp_df = spark.createDataFrame(lang_df)
sp_df.show()

+------+----------+---+----+
|python|javascript|  r|java|
+------+----------+---+----+
|   yes|        no|yes|  no|
|   yes|       yes|yes| yes|
+------+----------+---+----+



### -View the schema of the dataframe

In [6]:
sp_df.printSchema()

root
 |-- python: string (nullable = true)
 |-- javascript: string (nullable = true)
 |-- r: string (nullable = true)
 |-- java: string (nullable = true)



### -Output the shape of the dataframe

In [7]:
print((sp_df.count(), len(sp_df.columns)))

(2, 4)


### -Show the first 5 records in the dataframe

In [8]:
sp_df.show(5)

+------+----------+---+----+
|python|javascript|  r|java|
+------+----------+---+----+
|   yes|        no|yes|  no|
|   yes|       yes|yes| yes|
+------+----------+---+----+



***
## 2. Load the mpg dataset as a spark dataframe.

In [13]:
from pydataset import data

mpg_pd = data("mpg")
mpg_pd.head(5)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


### - A. Create 1 column of output that contains a message like the one below:
   `The 1999 audi a4 has a 4 cylinder engine.`

### - B. For each vehicle.
        - Transform the trans column so that it only contains either manual or auto.
 

***
## 3. Load the tips dataset as a spark dataframe.

### - A. What percentage of observations are smokers?
### - B. Create a column that contains the tip percentage
### - C. Calculate the average tip percentage for each combination of sex and smoker.

***
## 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

### -Convert the temperatures to farenheight.
### -Which month has the most rain, on average?
### -Which year was the windiest?
### -What is the most frequent type of weather in January?
### -What is the average high and low tempurature on sunny days in July in 2013 and 2014?
### -What percentage of days were rainy in q3 of 2015?
### -For each year, find what percentage of days it rained (had non-zero precipitation).

